In [16]:
import pandas as pd
import numpy as np
import math
import os
from tqdm import tqdm
from matplotlib import pyplot as plt
from PyRadioLoc.Utils.GeoUtils import GeoUtils
%run UtilsLightGBMFingerPrinting.ipynb
%run Imports.ipynb

# gera os resultados teoricos
for i in range(0,5):
    FILES = [
        (3*i + 13,'./k-fold-SVR/Test-SVR-Outdoor-Fold'+str(i+1)+'.csv'), # Usar CDB_20_SVR_Complete_k-fold 
        (3*i + 14,'./k-fold-SVR/Indoor_Test.csv'), # Usar CDB_20_SVR_Complete_k-fold
        (3*i + 15,'./k-fold-SVR/Test-SVR-Indoor-Outdoor-Fold'+str(i+1)+'.csv') # Usar CDB_20_SVR_Complete_k-fold
        ]
    TEST_DATABASE = "./k-fold-SVR/CDB-Fold"+str(i+1)+"-SVR.csv"
    generate_theorical_results(FILES, TEST_DATABASE)

FILES = [
    
    #k = 1 SVR
    './FinalResult/Resultados_Metodo_13.csv',  # CDB gerado com SVR dados Outdoor
    './FinalResult/Resultados_Metodo_14.csv',  # CDB gerado com SVR dados Indoor
    './FinalResult/Resultados_Metodo_15.csv',  # CDB gerado com SVR dados Indoor-Outdoor
    
    #k = 2 SVR
    './FinalResult/Resultados_Metodo_16.csv',  # CDB gerado com SVR dados Outdoor
    './FinalResult/Resultados_Metodo_17.csv',  # CDB gerado com SVR dados Indoor
    './FinalResult/Resultados_Metodo_18.csv',  # CDB gerado com SVR dados Indoor-Outdoor
    
    #k = 3 SVR
    './FinalResult/Resultados_Metodo_19.csv',  # CDB gerado com SVR dados Outdoor
    './FinalResult/Resultados_Metodo_20.csv',  # CDB gerado com SVR dados Indoor
    './FinalResult/Resultados_Metodo_21.csv',  # CDB gerado com SVR dados Indoor-Outdoor
    
    #k = 4 SVR
    './FinalResult/Resultados_Metodo_22.csv',  # CDB gerado com SVR dados Outdoor
    './FinalResult/Resultados_Metodo_23.csv',  # CDB gerado com SVR dados Indoor
    './FinalResult/Resultados_Metodo_24.csv',  # CDB gerado com SVR dados Indoor-Outdoor
    
    #k = 5 SVR
    './FinalResult/Resultados_Metodo_25.csv',  # CDB gerado com SVR dados Outdoor
    './FinalResult/Resultados_Metodo_26.csv',  # CDB gerado com SVR dados Indoor
    './FinalResult/Resultados_Metodo_27.csv',  # CDB gerado com SVR dados Indoor-Outdoor
    
    
]
METHODS = [x.split("_")[-1].split(".")[0] for x in FILES]     # lista contendo os metodos que serão usados
db = pd.read_csv('databases/meds_pandas.csv')
erbs = pd.read_csv('databases/Bts.csv')
#errorLocation = pd.read_csv('result/erros.csv')
ERROS_CSV_FILE = './result-k-fold/erros-SVR-k-fold.csv' 

400it [00:24, 16.37it/s]
400it [00:22, 23.72it/s]
400it [00:24, 13.81it/s]
400it [00:25, 18.05it/s]
400it [00:22, 17.65it/s]
400it [00:25, 14.86it/s]
400it [00:25, 15.15it/s]
400it [00:22, 17.42it/s]
400it [00:24, 16.44it/s]
400it [00:25, 15.86it/s]
400it [00:23, 16.96it/s]
400it [00:23, 16.73it/s]
400it [00:26, 14.76it/s]
400it [00:23, 17.24it/s]
400it [00:24, 16.53it/s]


In [17]:
# Gera o dataframe contendo os erros
errors_df = calculate_theorical_errors()
round(errors_df.describe(), 2)

,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
count,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00,400.00
mean,43.64,73.23,59.01,45.33,71.69,58.37,42.75,88.40,67.02,44.44,86.17,64.81,42.55,85.46,67.81
std,38.55,36.49,40.75,40.17,34.91,39.63,39.06,25.21,40.04,39.83,24.52,37.26,38.81,22.26,38.14
min,0.90,17.44,1.19,1.87,17.44,2.55,0.44,32.13,1.30,0.74,32.13,1.94,0.65,32.13,2.61
25%,14.74,42.13,18.79,14.82,42.13,22.08,12.61,72.88,30.14,13.90,72.88,29.38,14.05,72.88,32.13
50%,29.46,86.13,54.28,31.28,86.13,54.76,28.57,86.13,72.88,30.04,86.13,72.88,28.97,83.71,72.88
75%,62.06,101.93,90.76,65.61,96.80,88.28,60.18,102.64,96.42,63.53,101.93,92.82,61.12,101.93,92.82
max,189.02,198.83,173.53,208.22,198.83,194.16,196.32,137.37,206.63,197.64,198.83,207.88,245.89,137.37,184.93


In [18]:
test_erro = round(errors_df.describe(), 2)
test_erros = errors_df

In [19]:
std_outdoor = (test_erros['13'].std() + test_erros['16'].std() + test_erros['19'].std() + test_erros['22'].std() + test_erros['25'].std())/5
std_outdoor

39.28233192784258

In [20]:
std_indoor = (test_erros['14'].std() + test_erros['17'].std() + test_erros['20'].std() + test_erros['23'].std() + test_erros['26'].std())/5
std_indoor

28.67615413317963

In [21]:
std_indoor_outdoor = (test_erros['15'].std() + test_erros['18'].std() + test_erros['21'].std() + test_erros['24'].std() + test_erros['27'].std())/5
std_indoor_outdoor

39.165284177201016

# SVR-Outdoor:

In [22]:
test_erro = round(errors_df.describe(), 2)
test_erros = errors_df
mean_outdoor = (test_erros['13'] + test_erros['16'] + test_erros['19'] + test_erros['22'] + test_erros['25'])/5
mean_outdoor.head()

0    47.502
1    52.140
2    43.360
3    26.754
4    54.154
dtype: float64

In [23]:
min_outdoor = min(test_erro['13']['min'], test_erro['16']['min'], test_erro['19']['min'], test_erro['22']['min'], test_erro['25']['min'])
min_outdoor

0.44

In [24]:
max_outdoor = max(test_erro['13']['max'], test_erro['16']['max'], test_erro['19']['max'], test_erro['22']['max'], test_erro['25']['max'])
max_outdoor

245.89

# SVR-Indoor:

In [25]:
mean_indoor = (test_erros['14'] + test_erros['17'] + test_erros['20'] + test_erros['23'] + test_erros['26'])/5
#mean_indoor.head()
min_indoor = min(test_erro['14']['min'], test_erro['17']['min'], test_erro['20']['min'], test_erro['23']['min'], test_erro['26']['min'])
#min_indoor
max_indoor = max(test_erro['14']['max'], test_erro['17']['max'], test_erro['20']['max'], test_erro['23']['max'], test_erro['26']['max'])
#max_indoor
mean_indoor_outdoor = (test_erros['15'] + test_erros['18'] + test_erros['21'] + test_erros['24'] + test_erros['27'])/5
#mean_indoor_outdoor.head()
min_indoor_outdoor = min(test_erro['15']['min'], test_erro['18']['min'], test_erro['21']['min'], test_erro['24']['min'], test_erro['27']['min'])
#min_indoor_outdoor
max_indoor_outdoor = max(test_erro['15']['max'], test_erro['18']['max'], test_erro['21']['max'], test_erro['24']['max'], test_erro['27']['max'])
#max_indoor_outdoor
erros_SVR = [mean_outdoor,mean_indoor, mean_indoor_outdoor ]
erros_SVR = pd.DataFrame(erros_SVR)
erros_SVR = erros_SVR.T
erros_SVR = round(erros_SVR,2)
erros_SVR.to_csv(ERROS_CSV_FILE, index=False)
erros_SVR.head()


,0,1,2
0,47.50,88.53,58.63
1,52.14,42.13,25.73
2,43.36,98.54,24.91
3,26.75,50.70,47.32
4,54.15,88.70,60.52


In [26]:
max_indoor

198.83

In [27]:
min_indoor

17.44

In [28]:
max_indoor_outdoor

207.88

In [29]:
min_indoor_outdoor

1.19

In [30]:
results_SVR = round(erros_SVR.describe(), 2)
results_SVR
#results_SVR.to_csv('./result-k-fold/Result-SVR-k-fold.csv', index=False)

,0,1,2
count,400.00,400.00,400.00
mean,43.74,80.99,63.40
std,17.07,26.57,28.43
min,10.98,32.13,11.84
25%,31.50,61.87,42.13
50%,42.44,85.65,59.66
75%,54.17,98.54,88.53
max,106.23,165.04,128.43
